In [12]:
!pip install regex
!pip install demoji
!pip install plotly
!pip install wordcloud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.0 MB/s eta 0:00:00


In [13]:
import pandas as pd
import re

import regex
import demoji

import numpy as np
from collections import Counter

import plotly.express as px
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

In [64]:
#Si se usa colab, ejecutar esta celda
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
import re
# Patron regex para identificar el comienzo de cada línea del txt con la fecha y la hora

def IniciaConFechaYHora(s):
    # Ejemplo: '[4/21/24, 12:08:32 AM] '
    patron = r'^\[\d{1,2}\/\d{1,2}\/\d{2}, \d{1,2}:\d{2}:\d{2}\s?[AP]M\] '
    resultado = re.match(patron, s)  # Verificar si cada línea del txt hace match con el patrón de fecha y hora
    if resultado:
        return True
    return False

def EncontrarMiembro(mensaje):
    return ': ' in mensaje

def ObtenerPartes(linea):
    # Ejemplo: '[4/21/24, 12:09:57 AM] Queen🖤: image omitted'
    splitLinea = linea.split('] ', 1)
    FechaHora = splitLinea[0][1:]  # Remover el '[' inicial y obtener '4/21/24, 12:09:57 AM'
    splitFechaHora = FechaHora.split(', ', 1)
    Fecha = splitFechaHora[0]      # '4/21/24'
    Hora = splitFechaHora[1]       # '12:09:57 AM'
    MensajeCompleto = splitLinea[1]  # 'Queen🖤: image omitted'

    if EncontrarMiembro(MensajeCompleto):
        splitMensaje = MensajeCompleto.split(': ', 1)  # Split only on the first occurrence of ': '
        Miembro = splitMensaje[0]         # 'Queen🖤'
        Mensaje = splitMensaje[1]  # 'image omitted'
    else:
        Miembro = None
        Mensaje = MensajeCompleto

    return Fecha, Hora, Miembro, Mensaje

# Ejemplo de uso
linea = '[4/21/24, 12:09:57 AM] John: image omitted'
print(ObtenerPartes(linea))  # ('4/21/24', '12:09:57 AM', 'Queen🖤', 'image omitted')


('4/21/24', '12:09:57 AM', 'John', 'image omitted')


In [66]:
# Leer el archivo txt descargado del chat de WhatsApp
RutaChat = '/content/drive/MyDrive/_chat.txt'

# Lista para almacenar los datos (Fecha, Hora, Miembro, Mensaje) de cada línea del txt
DatosLista = []
with open(RutaChat, encoding="utf-8") as fp:
    fp.readline() # Eliminar primera fila relacionada al cifrado de extremo a extremo
    Fecha, Hora, Miembro = None, None, None
    while True:
        linea = fp.readline()
        if not linea:
            break
        linea = linea.strip()
        if IniciaConFechaYHora(linea): # Si cada línea del txt coincide con el patrón fecha y hora
            Fecha, Hora, Miembro, Mensaje = ObtenerPartes(linea) # Obtener datos de cada línea del txt
            DatosLista.append([Fecha, Hora, Miembro, Mensaje])

# Convertir la lista con los datos a dataframe
df = pd.DataFrame(DatosLista, columns=['Fecha', 'Hora', 'Miembro', 'Mensaje'])

# Cambiar la columna Fecha a formato datetime
df['Fecha'] = pd.to_datetime(df['Fecha'], format="%m/%d/%y")

# Eliminar los posibles campos vacíos del dataframe
# y lo que no son mensajes como cambiar el asunto del grupo o agregar a alguien
df = df.dropna()

# Resetear el índice
df.reset_index(drop=True, inplace=True)
df

,Fecha,Hora,Miembro,Mensaje
0,2024-04-21,12:10:47 AM,John Narvaez,"Con la foto del grado, pero ella no se parece ..."
1,2024-04-21,12:10:59 AM,Queen🖤,🫠🫠
2,2024-04-21,12:11:20 AM,Queen🖤,🤭
3,2024-04-21,12:11:45 AM,John Narvaez,"Dios mío, estás muy diferente jajajaja"
4,2024-04-21,12:11:54 AM,Queen🖤,Jajajajaaaj a
...,...,...,...,...
24437,2024-07-24,1:19:56 PM,Queen🖤,Muy maluca?
24438,2024-07-24,1:20:09 PM,John Narvaez,Preciosa amor
24439,2024-07-24,1:20:20 PM,John Narvaez,Me enamore mássss
24440,2024-07-24,1:23:02 PM,Queen🖤,Amor yo me sentía super rara pero ud me hizo s...


In [67]:
def ObtenerEmojis(Mensaje):
    emoji_lista = []
    data = regex.findall(r'\X', Mensaje)  # Obtener lista de caracteres de cada mensaje
    for caracter in data:
        if demoji.replace(caracter) != caracter:
            emoji_lista.append(caracter)
    return emoji_lista

# Obtener la cantidad total de mensajes
total_mensajes = df.shape[0]

# Obtener la cantidad de archivos multimedia enviados
multimedia_mensajes = '297'

# Obtener la cantidad de emojis enviados
df['Emojis'] = df['Mensaje'].apply(ObtenerEmojis) # Se agrega columna 'Emojis'
emojis = sum(df['Emojis'].str.len())

# Obtener la cantidad de links enviados
url_patron = r'(https?://\S+)'
df['URLs'] = df.Mensaje.apply(lambda x: len(re.findall(url_patron, x))) # Se agrega columna 'URLs'
links = sum(df['URLs'])

# Obtener la cantidad de encuestas
encuestas = df[df['Mensaje'] == 'POLL:'].shape[0]

# Todos los datos pasarlo a diccionario
estadistica_dict = {'Tipo': ['Mensajes', 'Multimedia', 'Emojis', 'Links'],
        'Cantidad': [total_mensajes, multimedia_mensajes, emojis, links]
        }

#Convertir diccionario a dataframe
estadistica_df = pd.DataFrame(estadistica_dict, columns = ['Tipo', 'Cantidad'])

# Establecer la columna Tipo como índice
estadistica_df = estadistica_df.set_index('Tipo')
estadistica_df

,Cantidad
Tipo,
Mensajes,24442
Multimedia,297
Emojis,7772
Links,937


Emojis más usados

In [68]:
# Obtener emojis más usados y las cantidades en el chat del grupo del dataframe
emojis_lista = list([a for b in df.Emojis for a in b])
emoji_diccionario = dict(Counter(emojis_lista))
emoji_diccionario = sorted(emoji_diccionario.items(), key=lambda x: x[1], reverse=True)

# Convertir el diccionario a dataframe
emoji_df = pd.DataFrame(emoji_diccionario, columns=['Emoji', 'Cantidad'])

# Establecer la columna Emoji como índice
emoji_df = emoji_df.set_index('Emoji').head(10)

print('Número emojis únicos usados: ', len(emoji_df), '\n')
emoji_df

Número emojis únicos usados:  10 



,Cantidad
Emoji,
🤣,1388
🤭,1280
💙,705
😬,522
🥰,410
🖤,362
☺️,252
🙈,239
😭,213


In [69]:
# Plotear el pie de los emojis más usados
fig = px.pie(emoji_df, values='Cantidad', names=emoji_df.index, hole=.3, template='plotly_dark', color_discrete_sequence=px.colors.qualitative.Pastel2)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(title={'text': '🤗 Emojis que más usamos', 'y':0.96, 'x':0.5, 'xanchor': 'center'}, font=dict(size=17))
fig.show()

Miembros


In [70]:
# Determinar los miembros más activos del grupo
df_MiembrosActivos = df.groupby('Miembro')['Mensaje'].count().sort_values(ascending=False).to_frame()
df_MiembrosActivos.reset_index(inplace=True)
df_MiembrosActivos.index = np.arange(1, len(df_MiembrosActivos)+1)
df_MiembrosActivos['% Mensaje'] = (df_MiembrosActivos['Mensaje'] / df_MiembrosActivos['Mensaje'].sum()) * 100
df_MiembrosActivos

,Miembro,Mensaje,% Mensaje
1,Queen🖤,13464,55.085509
2,John Narvaez,10978,44.914491


In [71]:
# Separar mensajes (sin multimedia) y multimedia (stickers, fotos, videos)
multimedia_df = df[df['Mensaje'] == '<Media omitted>']
mensajes_df = df.drop(multimedia_df.index)

# Contar la cantidad de palabras y letras por mensaje
mensajes_df['Letras'] = mensajes_df['Mensaje'].apply(lambda s : len(s))
mensajes_df['Palabras'] = mensajes_df['Mensaje'].apply(lambda s : len(s.split(' ')))
mensajes_df.tail()

,Fecha,Hora,Miembro,Mensaje,Emojis,URLs,Letras,Palabras
24437,2024-07-24,1:19:56 PM,Queen🖤,Muy maluca?,[],0,11,2
24438,2024-07-24,1:20:09 PM,John Narvaez,Preciosa amor,[],0,13,2
24439,2024-07-24,1:20:20 PM,John Narvaez,Me enamore mássss,[],0,17,3
24440,2024-07-24,1:23:02 PM,Queen🖤,Amor yo me sentía super rara pero ud me hizo s...,[],0,65,13
24441,2024-07-24,1:28:25 PM,John Narvaez,Eso me alegra amor,[],0,18,4


In [72]:
# Obtener a todos los miembros
miembros = mensajes_df.Miembro.unique()

# Crear diccionario donde se almacenará todos los datos
dictionario = {}

for i in range(len(miembros)):
    lista = []
    # Filtrar mensajes de un miembro en específico
    miembro_df= mensajes_df[mensajes_df['Miembro'] == miembros[i]]

    # Agregar a la lista el número total de mensajes enviados
    lista.append(miembro_df.shape[0])

    # Agregar a la lista el número de palabras por total de mensajes (palabras por mensaje)
    palabras_por_msj = (np.sum(miembro_df['Palabras']))/miembro_df.shape[0]
    lista.append(palabras_por_msj)

    # Agregar a la lista el número de mensajes multimedia enviados
    multimedia = multimedia_df[multimedia_df['Miembro'] == miembros[i]].shape[0]
    lista.append(multimedia)

    # Agregar a la lista el número total de emojis enviados
    emojis = sum(miembro_df['Emojis'].str.len())
    lista.append(emojis)

    # Agregar a la lista el número total de links enviados
    links = sum(miembro_df['URLs'])
    lista.append(links)

    # Asignar la lista como valor a la llave del diccionario
    dictionario[miembros[i]] = lista

print(dictionario)

{'John Narvaez': [10978, 8.557934049918018, 0, 2261, 615], 'Queen🖤': [13464, 7.94281045751634, 0, 5511, 322]}


In [73]:
# Convertir de diccionario a dataframe
miembro_stats_df = pd.DataFrame.from_dict(dictionario)

# Cambiar el índice por la columna agregada 'Estadísticas'
estadísticas = ['Mensajes', 'Palabras por mensaje', 'Multimedia', 'Emojis', 'Links']
miembro_stats_df['Estadísticas'] = estadísticas
miembro_stats_df.set_index('Estadísticas', inplace=True)

# Transponer el dataframe
miembro_stats_df = miembro_stats_df.T

#Convertir a integer las columnas Mensajes, Multimedia Emojis y Links
miembro_stats_df['Mensajes'] = miembro_stats_df['Mensajes'].apply(int)
miembro_stats_df['Multimedia'] = miembro_stats_df['Multimedia'].apply(int)
miembro_stats_df['Emojis'] = miembro_stats_df['Emojis'].apply(int)
miembro_stats_df['Links'] = miembro_stats_df['Links'].apply(int)
miembro_stats_df = miembro_stats_df.sort_values(by=['Mensajes'], ascending=False)
miembro_stats_df.drop(columns=['Multimedia'], inplace=True)
miembro_stats_df

Estadísticas,Mensajes,Palabras por mensaje,Emojis,Links
Queen🖤,13464,7.942810,5511,322
John Narvaez,10978,8.557934,2261,615


Estadisticas del comportamiento del chat

In [75]:
df['rangoHora'] = pd.to_datetime(df['Hora'], format='%I:%M:%S %p') # Include seconds in the format string

# Define a function to create the "Range Hour" column
def create_range_hour(hour):
    hour = pd.to_datetime(hour)  # Convertir a objeto de Python datetime si es necesario
    start_hour = hour.hour
    end_hour = (hour + pd.Timedelta(hours=1)).hour
    return f'{start_hour:02d} - {end_hour:02d} h'

# # Apply the function to create the "Range Hour" column
df['rangoHora'] = df['rangoHora'].apply(create_range_hour)
df

,Fecha,Hora,Miembro,Mensaje,Emojis,URLs,rangoHora
0,2024-04-21,12:10:47 AM,John Narvaez,"Con la foto del grado, pero ella no se parece ...",[],0,00 - 01 h
1,2024-04-21,12:10:59 AM,Queen🖤,🫠🫠,[],0,00 - 01 h
2,2024-04-21,12:11:20 AM,Queen🖤,🤭,[🤭],0,00 - 01 h
3,2024-04-21,12:11:45 AM,John Narvaez,"Dios mío, estás muy diferente jajajaja",[],0,00 - 01 h
4,2024-04-21,12:11:54 AM,Queen🖤,Jajajajaaaj a,[],0,00 - 01 h
...,...,...,...,...,...,...,...
24437,2024-07-24,1:19:56 PM,Queen🖤,Muy maluca?,[],0,13 - 14 h
24438,2024-07-24,1:20:09 PM,John Narvaez,Preciosa amor,[],0,13 - 14 h
24439,2024-07-24,1:20:20 PM,John Narvaez,Me enamore mássss,[],0,13 - 14 h
24440,2024-07-24,1:23:02 PM,Queen🖤,Amor yo me sentía super rara pero ud me hizo s...,[],0,13 - 14 h


In [76]:
df['DiaSemana'] = df['Fecha'].dt.strftime('%A')
mapeo_dias_espanol = {'Monday': '1 Lunes','Tuesday': '2 Martes','Wednesday': '3 Miércoles','Thursday': '4 Jueves',
                      'Friday': '5 Viernes','Saturday': '6 Sábado','Sunday': '7 Domingo'}
df['DiaSemana'] = df['DiaSemana'].map(mapeo_dias_espanol)
df

,Fecha,Hora,Miembro,Mensaje,Emojis,URLs,rangoHora,DiaSemana
0,2024-04-21,12:10:47 AM,John Narvaez,"Con la foto del grado, pero ella no se parece ...",[],0,00 - 01 h,7 Domingo
1,2024-04-21,12:10:59 AM,Queen🖤,🫠🫠,[],0,00 - 01 h,7 Domingo
2,2024-04-21,12:11:20 AM,Queen🖤,🤭,[🤭],0,00 - 01 h,7 Domingo
3,2024-04-21,12:11:45 AM,John Narvaez,"Dios mío, estás muy diferente jajajaja",[],0,00 - 01 h,7 Domingo
4,2024-04-21,12:11:54 AM,Queen🖤,Jajajajaaaj a,[],0,00 - 01 h,7 Domingo
...,...,...,...,...,...,...,...,...
24437,2024-07-24,1:19:56 PM,Queen🖤,Muy maluca?,[],0,13 - 14 h,3 Miércoles
24438,2024-07-24,1:20:09 PM,John Narvaez,Preciosa amor,[],0,13 - 14 h,3 Miércoles
24439,2024-07-24,1:20:20 PM,John Narvaez,Me enamore mássss,[],0,13 - 14 h,3 Miércoles
24440,2024-07-24,1:23:02 PM,Queen🖤,Amor yo me sentía super rara pero ud me hizo s...,[],0,13 - 14 h,3 Miércoles


Número de mensajes por rango de hora

In [79]:
# Crear una columna de 1 para realizar el conteo de mensajes
df['# Mensajes por hora'] = 1

# Sumar (contar) los mensajes que tengan la misma fecha
mensajes_hora = df.groupby('rangoHora').count().reset_index()

# Plotear la cantidad de mensajes respecto del tiempo
fig = px.line(mensajes_hora, x='rangoHora', y='# Mensajes por hora', color_discrete_sequence=['salmon'], template='plotly_dark')

# Ajustar el gráfico
fig.update_layout(
    title={'text': 'Mensajes con Queen🖤 por hora',
             'y':0.96,
             'x':0.5,
             'xanchor': 'center'},
    font=dict(size=17))
fig.update_traces(mode='markers+lines', marker=dict(size=10))
fig.update_xaxes(title_text='Rango de hora', tickangle=30)
fig.update_yaxes(title_text='# Mensajes')
fig.show()

Mensajes por día

In [80]:
# Crear una columna de 1 para realizar el conteo de mensajes
df['# Mensajes por día'] = 1

# Sumar (contar) los mensajes que tengan la misma fecha
date_df = df.groupby('DiaSemana').count().reset_index()


# Plotear la cantidad de mensajes respecto del tiempo
fig = px.line(date_df, x='DiaSemana', y='# Mensajes por día', color_discrete_sequence=['salmon'], template='plotly_dark')

# Ajustar el gráfico
# fig.update_layout(
#     title={'text': 'Mensajes con ella ❤️ por día', 'y':0.96, 'x':0.5, 'xanchor': 'center'},
#     font=dict(size=17))
fig.update_traces(mode='markers+lines', marker=dict(size=10))
fig.update_xaxes(title_text='Día', tickangle=30)
fig.update_yaxes(title_text='# Mensajes')
fig.show()

In [81]:
df.groupby('DiaSemana').count().reset_index()
# df.groupby(['DiaSemana']).size().reset_index(name='total_mensajes')


,DiaSemana,Fecha,Hora,Miembro,Mensaje,Emojis,URLs,rangoHora,# Mensajes por hora,# Mensajes por día
0,1 Lunes,3850,3850,3850,3850,3850,3850,3850,3850,3850
1,2 Martes,4473,4473,4473,4473,4473,4473,4473,4473,4473
2,3 Miércoles,3527,3527,3527,3527,3527,3527,3527,3527,3527
3,4 Jueves,3625,3625,3625,3625,3625,3625,3625,3625,3625
4,5 Viernes,4186,4186,4186,4186,4186,4186,4186,4186,4186
5,6 Sábado,1559,1559,1559,1559,1559,1559,1559,1559,1559
6,7 Domingo,3222,3222,3222,3222,3222,3222,3222,3222,3222


Mensajes a travez del tiempo

In [82]:
# Crear una columna de 1 para realizar el conteo de mensajes
df['# Mensajes por día'] = 1

# Sumar (contar) los mensajes que tengan la misma fecha
date_df = df.groupby('Fecha').sum().reset_index()

# Plotear la cantidad de mensajes respecto del tiempo
fig = px.line(date_df, x='Fecha', y='# Mensajes por día', color_discrete_sequence=['salmon'], template='plotly_dark')

# Ajustar el gráfico
# fig.update_layout(
#     title={'text': 'Mensajes con ella ❤️',
#              'y':0.96,
#              'x':0.5,
#              'xanchor': 'center'},
#     font=dict(size=17))

fig.update_xaxes(title_text='Fecha', tickangle=45, nticks=35)
fig.update_yaxes(title_text='# Mensajes')
fig.show()

Word cloud palabras más usadas

In [88]:
!pip install Pillow

In [89]:
from PIL import Image

In [90]:
from PIL import Image
import numpy as np

In [91]:
!ls /content/drive/MyDrive/
from google.colab import drive
drive.mount('/content/drive')

 005-HOJA-DE-VIDA-DOCENTES.gdoc  'Documento sin título.gdoc'
'Anexos Trabajo final.gdoc'	 'Examen 1 - John Jairo Narvaez Galviz.gdoc'
'cedula .gdoc'			  heart.jpg
 _chat.txt			 'Hoja de cálculo sin título.gsheet'
'Colab Notebooks'		  IMG_8097.jpeg
 curso_python			 'Jam sin título.gjam'
'Documento sin título (1).gdoc'  'mxc-wuiz-wxk - 16 de jun. de 2021.gjam'
'Documento sin título (2).gdoc'  'My city.pptx'
'Documento sin título (3).gdoc'  'Programa de Inducción 2018-I.pdf'
'Documento sin título (4).gdoc'  'Taller 2 - John Jairo Narvaez galviz.gdoc'
'Documento sin título (5).gdoc'  'Trabajo Final Inferencia.gslides'
'Documento sin título (6).gdoc'   y9
'Documento sin título (7).gdoc'
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Crear un string que contendrá todas las palabras
total_palabras = ' '
stopwords = STOPWORDS.update(['que', 'qué', 'con', 'de', 'te', 'en', 'la', 'lo', 'le', 'el', 'las', 'los', 'les', 'por', 'es',
                              'son', 'se', 'para', 'un', 'una', 'chicos', 'su', 'si', 'chic','nos', 'ya', 'hay', 'esta',
                              'pero', 'del', 'mas', 'más', 'eso', 'este', 'como', 'así', 'todo', 'https','Media','omitted',
                              'y', 'mi', 'o', 'q', 'yo', 'al', 'bien', 'vm','bueno'])

mask = np.array(Image.open('/heart.jpg'))

# Obtener y acumular todas las palabras de cada mensaje
for mensaje in mensajes_df['Mensaje'].values:
    palabras = str(mensaje).lower().split() # Obtener las palabras de cada línea del txt
    for palabra in palabras:
        total_palabras = total_palabras + palabra + ' ' # Acumular todas las palabras

wordcloud = WordCloud(width = 500, height = 500, background_color ='black', stopwords = stopwords,
                      max_words=100, min_font_size = 5,
                      mask = mask, colormap='OrRd',).generate(total_palabras)

# Plotear la nube de palabras más usadas
wordcloud.to_image()